In [30]:
# Nhiệm vụ 3 – Sử dụng GroupBy & Tổng hợp
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
from datetime import datetime




In [31]:
# Đọc dữ liệu từ các file CSV
owner_info = pd.read_csv('owner_info_sach.csv')
emission_test = pd.read_csv('emission_test_sach.csv')
vehicle_registration = pd.read_csv('vehicle_registration_sach.csv')


In [32]:
# Chuyển đổi kiểu dữ liệu ngày tháng
emission_test['test_date'] = pd.to_datetime(emission_test['test_date'])
vehicle_registration['reg_date'] = pd.to_datetime(vehicle_registration['reg_date'])

print("Đã chuyển đổi kiểu dữ liệu ngày tháng thành công!")
print(f"\nEmission Test date range: {emission_test['test_date'].min()} đến {emission_test['test_date'].max()}")
print(f"Vehicle Registration date range: {vehicle_registration['reg_date'].min()} đến {vehicle_registration['reg_date'].max()}")


Đã chuyển đổi kiểu dữ liệu ngày tháng thành công!

Emission Test date range: 2022-01-17 00:00:00 đến 2023-12-29 00:00:00
Vehicle Registration date range: 2021-01-01 00:00:00 đến 2023-12-20 00:00:00


In [33]:
# Kết hợp các bảng dữ liệu
# Bước 1: Kết hợp vehicle_registration với owner_info
df_merged = vehicle_registration.merge(
    owner_info, 
    on='owner_id', 
    how='left'
)

# Bước 2: Kết hợp với emission_test
df_final = df_merged.merge(
    emission_test,
    on='plate_no',
    how='left'
)

print(f"Kích thước dữ liệu sau khi merge: {df_final.shape}")
print(f"\nCác cột: {list(df_final.columns)}")
print(f"\nMẫu dữ liệu:")
print(df_final.head(10))


Kích thước dữ liệu sau khi merge: (147, 10)

Các cột: ['plate_no', 'owner_id', 'vehicle_type', 'engine_cc', 'reg_date', 'owner_name', 'province', 'test_date', 'result', 'co_level']

Mẫu dữ liệu:
     plate_no owner_id vehicle_type  engine_cc   reg_date    owner_name  \
0  51F-899.26   OWN136     Ô tô con        250 2021-03-24   Hoang Van E   
1  51F-881.31   OWN123       Xe máy        150 2021-10-11    Pham Thi D   
2  30A-009.15   OWN150     Ô tô con        100 2022-11-21    Pham Thi D   
3  30A-889.81   OWN141       Xe máy        250 2022-06-07      Le Van C   
4  43A-309.65   OWN103       Xe máy        125 2022-09-26  Nguyen Van A   
5  43A-309.65   OWN103       Xe máy        125 2022-09-26  Nguyen Van A   
6  43A-744.01   OWN134       Xe máy        250 2021-03-10      Le Van C   
7  51F-488.03   OWN151       Xe máy        110 2023-05-05  Nguyen Van A   
8  51F-488.03   OWN151       Xe máy        110 2023-05-05  Nguyen Van A   
9  29A-304.04   OWN100       Xe máy        150 2023-07-

In [34]:
# Tính toán thời gian xử lý (resolution_hours)
# Thời gian từ ngày đăng ký đến ngày kiểm tra khí thải
df_final['resolution_days'] = (df_final['test_date'] - df_final['reg_date']).dt.days
df_final['resolution_hours'] = df_final['resolution_days'] * 24

# Loại bỏ các giá trị âm, NaN hoặc không hợp lệ (kiểm tra trước đăng ký)
print(f"Số lượng bản ghi trước khi lọc: {df_final.shape[0]}")
print(f"Số lượng bản ghi có test_date thiếu: {df_final['test_date'].isna().sum()}")
print(f"Số lượng bản ghi có resolution_days < 0: {(df_final['resolution_days'] < 0).sum()}")

# Lọc dữ liệu hợp lệ
df_final = df_final[(df_final['resolution_days'] >= 0) & (df_final['resolution_days'].notna())]

print(f"\nSố lượng bản ghi sau khi loại bỏ giá trị không hợp lệ: {df_final.shape[0]}")
print(f"\nThống kê thời gian xử lý (ngày):")
print(df_final['resolution_days'].describe())


Số lượng bản ghi trước khi lọc: 147
Số lượng bản ghi có test_date thiếu: 57
Số lượng bản ghi có resolution_days < 0: 30

Số lượng bản ghi sau khi loại bỏ giá trị không hợp lệ: 60

Thống kê thời gian xử lý (ngày):
count      60.000000
mean      401.650000
std       275.198654
min        32.000000
25%       153.500000
50%       338.500000
75%       598.000000
max      1063.000000
Name: resolution_days, dtype: float64


In [35]:
# ============================================
# NHIỆM VỤ 1: Tính số lượng phản ánh trung bình theo area × category
# (Áp dụng: Số lượng kiểm tra khí thải trung bình theo province × vehicle_type)
# ============================================

# Đếm số lượng kiểm tra theo province × vehicle_type
test_counts = df_final.groupby(['province', 'vehicle_type']).agg({
    'plate_no': 'count',  # Số lượng kiểm tra
    'test_date': 'count'  # Cũng có thể dùng cột này
}).rename(columns={'plate_no': 'so_luong_kiem_tra'})

# Tính trung bình số lượng kiểm tra theo từng nhóm
avg_tests = df_final.groupby(['province', 'vehicle_type']).size().reset_index(name='so_luong_kiem_tra')
avg_tests_by_area_category = avg_tests.groupby(['province', 'vehicle_type'])['so_luong_kiem_tra'].mean().reset_index()

print("=== SỐ LƯỢNG KIỂM TRA TRUNG BÌNH THEO PROVINCE × VEHICLE_TYPE ===")
print("\nTổng số lượng kiểm tra theo từng nhóm:")
print(test_counts)
print("\n" + "="*80)
print("\nSố lượng kiểm tra trung bình theo province × vehicle_type:")
print(avg_tests_by_area_category)
print("\n" + "="*80)
print("\nDạng pivot table để dễ đọc:")
pivot_avg = avg_tests.pivot_table(
    values='so_luong_kiem_tra',
    index='province',
    columns='vehicle_type',
    aggfunc='mean',
    fill_value=0
)
print(pivot_avg)


=== SỐ LƯỢNG KIỂM TRA TRUNG BÌNH THEO PROVINCE × VEHICLE_TYPE ===

Tổng số lượng kiểm tra theo từng nhóm:
                         so_luong_kiem_tra  test_date
province   vehicle_type                              
Bình Dương Xe máy                        7          7
           Xe tải                        1          1
           Ô tô con                      4          4
Hà Nội     Xe máy                        4          4
           Xe tải                        1          1
           Ô tô con                      6          6
TP.HCM     Xe máy                        4          4
           Xe tải                        4          4
           Ô tô con                      2          2
Đà Nẵng    Xe máy                       12         12
           Xe tải                       11         11
           Ô tô con                      4          4


Số lượng kiểm tra trung bình theo province × vehicle_type:
      province vehicle_type  so_luong_kiem_tra
0   Bình Dương       Xe máy   

In [36]:
# ============================================
# NHIỆM VỤ 2: Tính thời gian xử lý trung bình (resolution_hours) theo department
# (Áp dụng: Thời gian xử lý trung bình theo province)
# ============================================

resolution_by_province = df_final.groupby('province').agg({
    'resolution_hours': ['mean', 'median', 'std', 'min', 'max', 'count']
}).round(2)

resolution_by_province.columns = ['trung_binh_gio', 'trung_vi_gio', 'do_lech_chuan', 'toi_thieu', 'toi_da', 'so_luong']

print("=== THỜI GIAN XỬ LÝ TRUNG BÌNH THEO PROVINCE (DEPARTMENT) ===")
print(resolution_by_province.sort_values('trung_binh_gio', ascending=False))
print("\n" + "="*80)
print("\nThời gian xử lý trung bình (giờ) theo province:")
print(resolution_by_province[['trung_binh_gio', 'so_luong']].sort_values('trung_binh_gio', ascending=False))


=== THỜI GIAN XỬ LÝ TRUNG BÌNH THEO PROVINCE (DEPARTMENT) ===
            trung_binh_gio  trung_vi_gio  do_lech_chuan  toi_thieu   toi_da  \
province                                                                      
Hà Nội            12870.55       11640.0        7244.15     3648.0  25512.0   
Bình Dương        10728.00       10644.0        6771.66     2592.0  21432.0   
Đà Nẵng            8944.00        7272.0        6155.84     1104.0  21528.0   
TP.HCM             6657.60        4428.0        6039.53      768.0  17496.0   

            so_luong  
province              
Hà Nội            11  
Bình Dương        12  
Đà Nẵng           27  
TP.HCM            10  


Thời gian xử lý trung bình (giờ) theo province:
            trung_binh_gio  so_luong
province                            
Hà Nội            12870.55        11
Bình Dương        10728.00        12
Đà Nẵng            8944.00        27
TP.HCM             6657.60        10


In [37]:
# ============================================
# NHIỆM VỤ 3: Tính thời gian xử lý trung bình theo department × category
# (Áp dụng: Thời gian xử lý trung bình theo province × vehicle_type)
# ============================================

resolution_by_province_type = df_final.groupby(['province', 'vehicle_type']).agg({
    'resolution_hours': ['mean', 'median', 'std', 'count']
}).round(2)

resolution_by_province_type.columns = ['trung_binh_gio', 'trung_vi_gio', 'do_lech_chuan', 'so_luong']

print("=== THỜI GIAN XỬ LÝ TRUNG BÌNH THEO PROVINCE × VEHICLE_TYPE ===")
print(resolution_by_province_type.sort_values('trung_binh_gio', ascending=False))
print("\n" + "="*80)
print("\nDạng pivot table để dễ so sánh:")
pivot_resolution = df_final.pivot_table(
    values='resolution_hours',
    index='province',
    columns='vehicle_type',
    aggfunc='mean',
    fill_value=0
).round(2)
print(pivot_resolution)


=== THỜI GIAN XỬ LÝ TRUNG BÌNH THEO PROVINCE × VEHICLE_TYPE ===
                         trung_binh_gio  trung_vi_gio  do_lech_chuan  so_luong
province   vehicle_type                                                       
Hà Nội     Ô tô con            17300.00       17640.0        5767.48         6
Bình Dương Xe máy              13268.57       13200.0        6144.05         7
Đà Nẵng    Ô tô con            12066.00       14472.0        7664.19         4
Hà Nội     Xe tải              11640.00       11640.0            NaN         1
Đà Nẵng    Xe máy              10572.00       10272.0        7076.58        12
Bình Dương Ô tô con             7986.00        5544.0        7215.70         4
TP.HCM     Xe tải               7566.00        7332.0        4662.24         4
           Ô tô con             7500.00        7500.0        7280.37         2
Hà Nội     Xe máy               6534.00        4152.0        5119.09         4
Đà Nẵng    Xe tải               6032.73        6696.0        3068.4

In [38]:
# ============================================
# NHIỆM VỤ 4: So sánh thời gian xử lý trung bình giữa các channel tiếp nhận
# (Áp dụng: So sánh thời gian xử lý trung bình giữa các loại xe - vehicle_type)
# ============================================

resolution_by_vehicle_type = df_final.groupby('vehicle_type').agg({
    'resolution_hours': ['mean', 'median', 'std', 'min', 'max', 'count']
}).round(2)

resolution_by_vehicle_type.columns = ['trung_binh_gio', 'trung_vi_gio', 'do_lech_chuan', 'toi_thieu', 'toi_da', 'so_luong']

print("=== SO SÁNH THỜI GIAN XỬ LÝ TRUNG BÌNH THEO VEHICLE_TYPE (CHANNEL) ===")
print(resolution_by_vehicle_type.sort_values('trung_binh_gio', ascending=False))
print("\n" + "="*80)

# So sánh chi tiết hơn
print("\nThời gian xử lý trung bình (giờ) theo từng loại xe:")
for vehicle_type in df_final['vehicle_type'].unique():
    data = df_final[df_final['vehicle_type'] == vehicle_type]['resolution_hours']
    print(f"\n{vehicle_type}:")
    print(f"  - Trung bình: {data.mean():.2f} giờ ({data.mean()/24:.2f} ngày)")
    print(f"  - Trung vị: {data.median():.2f} giờ ({data.median()/24:.2f} ngày)")
    print(f"  - Số lượng: {len(data)}")


=== SO SÁNH THỜI GIAN XỬ LÝ TRUNG BÌNH THEO VEHICLE_TYPE (CHANNEL) ===
              trung_binh_gio  trung_vi_gio  do_lech_chuan  toi_thieu   toi_da  \
vehicle_type                                                                    
Ô tô con            12438.00       12648.0        7403.51     1104.0  25512.0   
Xe máy               9896.00        8904.0        6983.58      768.0  21528.0   
Xe tải               6598.59        6696.0        3529.00     2136.0  13440.0   

              so_luong  
vehicle_type            
Ô tô con            16  
Xe máy              27  
Xe tải              17  


Thời gian xử lý trung bình (giờ) theo từng loại xe:

Ô tô con:
  - Trung bình: 12438.00 giờ (518.25 ngày)
  - Trung vị: 12648.00 giờ (527.00 ngày)
  - Số lượng: 16

Xe máy:
  - Trung bình: 9896.00 giờ (412.33 ngày)
  - Trung vị: 8904.00 giờ (371.00 ngày)
  - Số lượng: 27

Xe tải:
  - Trung bình: 6598.59 giờ (274.94 ngày)
  - Trung vị: 6696.00 giờ (279.00 ngày)
  - Số lượng: 17


In [39]:
# ============================================
# NHIỆM VỤ 5: Xác định nhóm area hoặc department có thời gian xử lý cao bất thường
# ============================================

# Tính toán các chỉ số để xác định nhóm bất thường
# Sử dụng Z-score hoặc IQR để phát hiện outliers

# Tính thời gian xử lý trung bình theo province
province_stats = df_final.groupby('province')['resolution_hours'].agg(['mean', 'std', 'count']).reset_index()
province_stats.columns = ['province', 'mean_hours', 'std_hours', 'count']

# Tính Z-score
overall_mean = df_final['resolution_hours'].mean()
overall_std = df_final['resolution_hours'].std()
province_stats['z_score'] = (province_stats['mean_hours'] - overall_mean) / overall_std

# Xác định nhóm bất thường (Z-score > 1.5 hoặc < -1.5)
province_stats['bat_thuong'] = province_stats['z_score'].abs() > 1.5
province_stats['loai_bat_thuong'] = province_stats.apply(
    lambda x: 'Cao bất thường' if x['z_score'] > 1.5 
    else 'Thấp bất thường' if x['z_score'] < -1.5 
    else 'Bình thường', axis=1
)

print("=== PHÁT HIỆN NHÓM CÓ THỜI GIAN XỬ LÝ BẤT THƯỜNG ===")
print(f"\nThời gian xử lý trung bình tổng thể: {overall_mean:.2f} giờ ({overall_mean/24:.2f} ngày)")
print(f"Độ lệch chuẩn tổng thể: {overall_std:.2f} giờ")
print("\n" + "="*80)
print("\nThống kê theo province:")
print(province_stats.sort_values('mean_hours', ascending=False))
print("\n" + "="*80)
print("\nCác nhóm có thời gian xử lý CAO BẤT THƯỜNG (Z-score > 1.5):")
high_anomalies = province_stats[province_stats['z_score'] > 1.5].sort_values('z_score', ascending=False)
if len(high_anomalies) > 0:
    print(high_anomalies[['province', 'mean_hours', 'z_score', 'count']])
else:
    print("Không có nhóm nào có thời gian xử lý cao bất thường (Z-score > 1.5)")
print("\n" + "="*80)
print("\nCác nhóm có thời gian xử lý THẤP BẤT THƯỜNG (Z-score < -1.5):")
low_anomalies = province_stats[province_stats['z_score'] < -1.5].sort_values('z_score', ascending=True)
if len(low_anomalies) > 0:
    print(low_anomalies[['province', 'mean_hours', 'z_score', 'count']])
else:
    print("Không có nhóm nào có thời gian xử lý thấp bất thường (Z-score < -1.5)")


=== PHÁT HIỆN NHÓM CÓ THỜI GIAN XỬ LÝ BẤT THƯỜNG ===

Thời gian xử lý trung bình tổng thể: 9639.60 giờ (401.65 ngày)
Độ lệch chuẩn tổng thể: 6604.77 giờ


Thống kê theo province:
     province    mean_hours    std_hours  count   z_score  bat_thuong  \
1      Hà Nội  12870.545455  7244.145393     11  0.489184       False   
0  Bình Dương  10728.000000  6771.658586     12  0.164790       False   
3     Đà Nẵng   8944.000000  6155.840754     27 -0.105318       False   
2      TP.HCM   6657.600000  6039.530578     10 -0.451492       False   

  loai_bat_thuong  
1     Bình thường  
0     Bình thường  
3     Bình thường  
2     Bình thường  


Các nhóm có thời gian xử lý CAO BẤT THƯỜNG (Z-score > 1.5):
Không có nhóm nào có thời gian xử lý cao bất thường (Z-score > 1.5)


Các nhóm có thời gian xử lý THẤP BẤT THƯỜNG (Z-score < -1.5):
Không có nhóm nào có thời gian xử lý thấp bất thường (Z-score < -1.5)


In [40]:
# Phân tích bất thường theo province × vehicle_type
province_type_stats = df_final.groupby(['province', 'vehicle_type'])['resolution_hours'].agg(['mean', 'std', 'count']).reset_index()
province_type_stats.columns = ['province', 'vehicle_type', 'mean_hours', 'std_hours', 'count']

# Tính Z-score cho từng nhóm
province_type_stats['z_score'] = (province_type_stats['mean_hours'] - overall_mean) / overall_std
province_type_stats['bat_thuong'] = province_type_stats['z_score'].abs() > 1.5
province_type_stats['loai_bat_thuong'] = province_type_stats.apply(
    lambda x: 'Cao bất thường' if x['z_score'] > 1.5 
    else 'Thấp bất thường' if x['z_score'] < -1.5 
    else 'Bình thường', axis=1
)

print("\n" + "="*80)
print("=== PHÁT HIỆN NHÓM CÓ THỜI GIAN XỬ LÝ BẤT THƯỜNG (PROVINCE × VEHICLE_TYPE) ===")
print("\nCác nhóm có thời gian xử lý CAO BẤT THƯỜNG:")
high_anomalies_detail = province_type_stats[province_type_stats['z_score'] > 1.5].sort_values('z_score', ascending=False)
if len(high_anomalies_detail) > 0:
    print(high_anomalies_detail[['province', 'vehicle_type', 'mean_hours', 'z_score', 'count']])
else:
    print("Không có nhóm nào có thời gian xử lý cao bất thường")
    
print("\nTop 5 nhóm có thời gian xử lý cao nhất:")
print(province_type_stats.nlargest(5, 'mean_hours')[['province', 'vehicle_type', 'mean_hours', 'z_score', 'count']])



=== PHÁT HIỆN NHÓM CÓ THỜI GIAN XỬ LÝ BẤT THƯỜNG (PROVINCE × VEHICLE_TYPE) ===

Các nhóm có thời gian xử lý CAO BẤT THƯỜNG:
Không có nhóm nào có thời gian xử lý cao bất thường

Top 5 nhóm có thời gian xử lý cao nhất:
      province vehicle_type    mean_hours   z_score  count
5       Hà Nội     Ô tô con  17300.000000  1.159829      6
0   Bình Dương       Xe máy  13268.571429  0.549447      7
11     Đà Nẵng     Ô tô con  12066.000000  0.367371      4
4       Hà Nội       Xe tải  11640.000000  0.302872      1
9      Đà Nẵng       Xe máy  10572.000000  0.141171     12
